In [1]:
from pyspark.sql import SparkSession
import psycopg2

In [2]:
packages = {
   'org.postgresql:postgresql:42.6.0'
}

In [3]:
spark = (
    SparkSession.builder
    .appName('postgres')
    .config('spark.jars.packages', ','.join(packages))
    .getOrCreate()    
)

:: loading settings :: url = jar:file:/home/hduser/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hduser/.ivy2/cache
The jars for the packages stored in: /home/hduser/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-58d313fe-4fce-4dbe-84c8-a88500fe27de;1.0
	confs: [default]
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 207ms :: artifacts dl 9ms
	:: modules in use:
	org.checkerframework#checker-qual;3.31.0 from central in [default]
	org.postgresql#postgresql;42.6.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-------------------------------------------

23/04/21 07:06:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
heroes = spark.createDataFrame([
        ("Bilbo Baggins", 50),
        ("Gandalf", 1000),
        ("Thorin", 195),
        ("Balin", 178),
        ("Kili", 77),
        ("Dwalin", 169),
        ("Oin", 167),
        ("Gloin", 158),
        ("Fili", 82),
        ("Bombur", None),
    ],
    ["name", "age"],
)

In [5]:
options = {
    'format': 'jdbc',
    'url': 'jdbc:postgresql://postgres:5432/postgres',
    'driver': 'org.postgresql.Driver',
    'user': 'postgres',
    'password': 'postgres',
}

In [6]:
heroes.write.mode('overwrite').options(**options, dbtable='heroes', path='heroes').save()

In [7]:
df = spark.read.options(**options, dbtable = heroes, path='heroes').load()

In [8]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [9]:
conn = psycopg2.connect(
    host='postgres',
    database='postgres',
    user='postgres',
    password='postgres'
)
cur = conn.cursor()
with open('/workspace/assignments/data/northwind.sql', 'r') as f:
    cur.execute(f.read())
    
conn.commit()
cur.close()
conn.close()

In [10]:
df = spark.read.load(**options, dbtable='categories')
df.printSchema()

root
 |-- category_id: short (nullable = true)
 |-- category_name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- picture: binary (nullable = true)



In [11]:
df.show()

+-----------+--------------+--------------------+-------+
|category_id| category_name|         description|picture|
+-----------+--------------+--------------------+-------+
|          1|     Beverages|Soft drinks, coff...|     []|
|          2|    Condiments|Sweet and savory ...|     []|
|          3|   Confections|Desserts, candies...|     []|
|          4|Dairy Products|             Cheeses|     []|
|          5|Grains/Cereals|Breads, crackers,...|     []|
|          6|  Meat/Poultry|      Prepared meats|     []|
|          7|       Produce|Dried fruit and b...|     []|
|          8|       Seafood|    Seaweed and fish|     []|
+-----------+--------------+--------------------+-------+

